# DEVELOPING THE MAZE GENERATOR

Importing the libraries:

In [1]:
import numpy as np

Defining auxiliary classes:

In [2]:
class UnionFind:
    def __init__(self, n):
        self.n = n  # number of supernodes
        self.v = [i for i in range(n)]  # represents the parents (initially each disjoint set has a single vertex)

    def find(self, u):
        while u != self.v[u]:
            self.v[u] = self.v[self.v[u]]  # compression technique
            u = self.v[u]
            
        return u
    
    def union(self, u, v):
        root_u, root_v = self.find(u), self.find(v)
        
        if root_u == root_v:
            return False  # union was not performed
        else:
            self.v[root_v] = root_u
            self.n -= 1

            return True  # union was not performed

Defining the maze generator:

 - Labyrinth is represented by an adjacency list, as the resulting graph will always be sparse

In [3]:
# Class for the maze generator
# Inspired by a randomized Kruskal algorithm
class MazeGenerator:
    def __init__(self, N=100):
        self.N = N  # number os cells (default is a 10x10 grid)
        
        self.walls = self.generate_walls(self.N)
        
    def new_maze(self):
        maze = [[] for _ in range(self.N)]  # data structure to store the maze
        
        # Create a set for each cell and shuffle list of walls (edges)
        forest = UnionFind(self.N)
        
        walls = self.walls.copy()
        np.random.shuffle(walls)
        
        # Generate a maze (represented by an adjacency list)
        while forest.n > 1:
            v, w = walls.pop(np.random.randint(0, len(walls)))
            
            if forest.union(v, w):
                maze[v].append(w)  # adding (v,w) in the maze
                maze[w].append(v)  # adding (w,v) in the maze
                
        return maze
                
    @staticmethod
    def actions(N, v):
        maze_shape = int(N**0.5)

        actions = []
        if v % maze_shape != 0:
            actions.append((v, v-1))
        if (v+1) % maze_shape != 0:
            actions.append((v, v+1))
        if v >= maze_shape:
            actions.append((v, v-maze_shape))
        if v+maze_shape < N:
            actions.append((v, v+maze_shape))

        return actions  # possible actions in cell v

    def generate_walls(self, N):
        walls = []
        
        # Movement represents the existence or not of a wall
        for v in range(N):
            walls.extend(self.actions(N, v))

        return walls

Testing the generator:

In [4]:
# Using the default of 100 cells

generator = MazeGenerator()

maze = generator.new_maze()

maze

[[1],
 [11, 0, 2],
 [1],
 [13, 4],
 [3],
 [6, 15],
 [5],
 [17, 8],
 [9, 7],
 [8, 19],
 [11],
 [1, 12, 10],
 [11, 13],
 [23, 3, 14, 12],
 [24, 13, 15],
 [16, 5, 14],
 [26, 15],
 [7],
 [28],
 [29, 9],
 [21, 30],
 [20, 31],
 [23],
 [13, 22],
 [14],
 [26, 35],
 [16, 36, 25, 27],
 [26, 28],
 [18, 29, 27, 38],
 [19, 28],
 [20],
 [41, 21],
 [42],
 [34],
 [35, 33],
 [34, 25],
 [26, 37],
 [36],
 [48, 28],
 [49],
 [50],
 [31, 51],
 [52, 32],
 [44],
 [43, 54],
 [46],
 [45, 47],
 [46, 48, 57],
 [47, 38],
 [59, 39],
 [60, 40, 51],
 [52, 50, 41],
 [53, 42, 51, 62],
 [52, 54, 63],
 [55, 44, 53],
 [65, 54, 56],
 [57, 55],
 [58, 56, 47],
 [57, 68],
 [49, 69],
 [50, 70],
 [71],
 [72, 52],
 [53],
 [74],
 [55],
 [67],
 [68, 66, 77],
 [67, 58, 69],
 [59, 68],
 [71, 60, 80],
 [70, 61],
 [82, 73, 62],
 [72, 74],
 [64, 73],
 [85],
 [86],
 [87, 67],
 [88, 79],
 [78],
 [90, 70],
 [91, 82],
 [72, 92, 83, 81],
 [82],
 [85],
 [75, 86, 84],
 [76, 85, 96],
 [88, 77],
 [78, 98, 87],
 [99],
 [80],
 [81],
 [82, 93],
 [

Printing the results:

In [5]:
maze_shape = int(generator.N**0.5)

for row in range(maze_shape):
    for col in range(maze_shape-1):
        print('x', end='')
        
        if row*10+col in maze[row*10+col+1]:
            print(' ', end='')
        else:
            print('|', end='')
            
    print('x')
    
    if row != 9:
        for col in range(maze_shape):
            if row*10+col in maze[(row+1)*10+col]:
                print('  ', end='')
            else:
                print('_ ', end='')
                
        print()

x x x|x x|x x|x x x
_   _   _   _   _   
x x x x x x x|x|x|x
_ _ _     _   _     
x x|x x|x|x x x x x
    _ _ _     _   _ 
x|x|x|x x x|x x|x|x
_     _ _ _ _ _     
x|x|x|x x|x x x x|x
      _   _ _   _   
x x x x x x x x x|x
  _     _   _ _     
x|x|x|x|x|x|x x x x
      _   _ _   _ _ 
x x|x x x|x|x|x|x x
  _   _ _         _ 
x|x x x|x x x|x x|x
      _ _ _   _     
x|x|x x x x x|x x x
